# Unified Clinical Corpus → Chroma Pipeline

This notebook loads the first 1,000 rows from `medical_data.csv`, `patient_notes.csv`, `PMC-Patients.csv`, and `pubmed_dataset.csv`, cleans and de-identifies every note, merges all text into a single master document, and stores both the full document and LangChain-tokenised passages inside a Chroma vector database backed by BioClinicalBERT embeddings.

Workflow outline:
1. **Cleaning & Normalisation** – remove noise/special symbols, expand UMLS-style medical abbreviations, lowercase, and redact PHI markers for HIPAA/GDPR compliance.
2. **LangChain Tokenisation & BioClinicalBERT Embeddings** – build a master document, measure token counts with a Hugging Face tokenizer, and prepare embeddings with a PubMed/BioBERT-family model.
3. **Chunking & ChromaDB Storage** – split the master document into ~100–300 word passages using `RecursiveCharacterTextSplitter`, then insert both the full document and the chunks into a Chroma collection (metadata includes source, chunk index, timestamps).
4. **Final Outputs** – persist the cleaned corpus, chunk metadata, Chroma persistence directory, and demonstrate a sample semantic query for RAG readiness.



In [ ]:
import os
import re
import shutil
from collections import defaultdict
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

import nltk
nltk.download('punkt', quiet=True)

import spacy
from scispacy.abbreviation import AbbreviationDetector

# Updated imports: Always use explicit import; dependencies should be pre-installed as per instructions.
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Chroma & Embeddings config – for downstream cells:
EMBEDDING_MODEL_NAME = "pritamdeka/pubmedbert-base-embeddings"
CHROMA_PERSIST_DIR = Path("data/processed/chroma_clinical/").resolve()
CHUNK_METADATA_PATH = Path("data/processed/chunk_metadata.parquet").resolve()


## Paths & Dataset Configuration

We pull all source data from `data/raw/` and constrain to the first 1,000 rows per file to keep the demonstration tractable.


In [11]:
DATA_ROOT = Path('/home/root495/Inexture/CDSS-RAG/data/raw')
OUTPUT_ROOT = Path('/home/root495/Inexture/CDSS-RAG/data/processed')
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

DATASETS = {
    'medical_data': {'path': DATA_ROOT / 'medical_data.csv', 'text_column': 'TEXT'},
    'patient_notes': {'path': DATA_ROOT / 'patient_notes.csv', 'text_column': 'pn_history'},
    'pmc_patients': {'path': DATA_ROOT / 'PMC-Patients.csv', 'text_column': 'patient'},
    'pubmed': {'path': DATA_ROOT / 'pubmed_dataset.csv', 'text_column': 'contents'}
}

MAX_ROWS = 1000
DATASETS


{'medical_data': {'path': PosixPath('/home/root495/Inexture/CDSS-RAG/data/raw/medical_data.csv'),
  'text_column': 'TEXT'},
 'patient_notes': {'path': PosixPath('/home/root495/Inexture/CDSS-RAG/data/raw/patient_notes.csv'),
  'text_column': 'pn_history'},
 'pmc_patients': {'path': PosixPath('/home/root495/Inexture/CDSS-RAG/data/raw/PMC-Patients.csv'),
  'text_column': 'patient'},
 'pubmed': {'path': PosixPath('/home/root495/Inexture/CDSS-RAG/data/raw/pubmed_dataset.csv'),
  'text_column': 'contents'}}

## Load First 1,000 Rows Per Dataset

Each dataset is truncated with `head(1000)` and tagged with `source` plus `source_row_id` for downstream metadata.


In [12]:
raw_dfs = {}
for name, cfg in DATASETS.items():
    df = pd.read_csv(cfg['path']).head(MAX_ROWS)
    df['source'] = name
    df['source_row_id'] = df.index
    raw_dfs[name] = df

{k: len(v) for k, v in raw_dfs.items()}


{'medical_data': 744,
 'patient_notes': 1000,
 'pmc_patients': 1000,
 'pubmed': 1000}

In [13]:
{name: df[[DATASETS[name]['text_column']]].head(2) for name, df in raw_dfs.items()}


{'medical_data':                                                 TEXT
 0  Admission Date:  [**2162-3-3**]              D...
 1  Admission Date:  [**2150-2-25**]              ...,
 'patient_notes':                                           pn_history
 0  17-year-old male, has come to the student heal...
 1  17 yo male with recurrent palpitations for the...,
 'pmc_patients':                                              patient
 0  This 60-year-old male was hospitalized due to ...
 1  A 39-year-old man was hospitalized due to an i...,
 'pubmed':                                             contents
 0  [Biochemical studies on camomile components/II...
 1  [Demonstration of tumor inhibiting properties ...}

## Cleaning & Normalisation

We remove noisy symbols, standardise spacing/case, expand common medical abbreviations (UMLS-inspired) via ScispaCy’s abbreviation detector, and redact PHI markers such as emails, phone numbers, MRNs, and dates.


In [14]:
NLP_MODEL = 'en_core_sci_sm'
nlp = spacy.load(NLP_MODEL, disable=['ner'])
nlp.add_pipe('abbreviation_detector')

UMLS_ABBREV_MAP = {
    'HTN': 'hypertension',
    'DM': 'diabetes mellitus',
    'SOB': 'shortness of breath',
    'CAD': 'coronary artery disease',
    'COPD': 'chronic obstructive pulmonary disease',
    'CHF': 'congestive heart failure',
    'Pt': 'patient',
    'BP': 'blood pressure',
    'HR': 'heart rate',
    'c/o': 'complains of'
}

PHI_PATTERNS = {
    'emails': re.compile(r'\b[\w.-]+@[\w.-]+\.[A-Za-z]{2,}\b'),
    'phones': re.compile(r'(?:\+?\d{1,2}[ -]?)?(?:\(\d{3}\)|\d{3})[ -]?\d{3}[ -]?\d{4}'),
    'dates': re.compile(r'\b(?:\d{1,2}[/-]){2}\d{2,4}\b'),
    'mrn': re.compile(r'\b(?:mrn|patient id)\s*[:#]?\s*\d+\b', re.IGNORECASE),
    'names': re.compile(r'\b([A-Z][a-z]+\s[A-Z][a-z]+)\b')
}


def normalize_whitespace(text: str) -> str:
    return re.sub(r'\s+', ' ', str(text)).strip()


def remove_special_chars(text: str) -> str:
    return re.sub(r"[^0-9A-Za-z.,;:!?%\-\s'\/]", ' ', text)


def expand_abbreviations(text: str) -> str:
    doc = nlp(text)
    expanded = text
    for abrv in doc._.abbreviations:
        key = abrv.text.strip()
        if key in UMLS_ABBREV_MAP:
            expanded = re.sub(rf'\b{re.escape(key)}\b', UMLS_ABBREV_MAP[key], expanded)
    for short, long in UMLS_ABBREV_MAP.items():
        expanded = re.sub(rf'\b{re.escape(short)}\b', long, expanded, flags=re.IGNORECASE)
    return expanded


def deidentify(text: str) -> str:
    redacted = text
    for pattern in PHI_PATTERNS.values():
        redacted = pattern.sub('[REDACTED]', redacted)
    return redacted


def clean_text(text: str) -> str:
    text = normalize_whitespace(text)
    text = remove_special_chars(text)
    text = expand_abbreviations(text)
    text = text.lower()
    text = deidentify(text)
    return text

cleaned_dfs = {}
for name, df in raw_dfs.items():
    text_col = DATASETS[name]['text_column']
    df = df.copy()x``
    df['clean_text'] = df[text_col].fillna('').apply(clean_text)
    cleaned_dfs[name] = df

{k: df[['clean_text']].head(1)['clean_text'].iloc[0][:120] for k, df in cleaned_dfs.items()}



/home/root495/.local/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/root495/.local/lib/python3.10/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/root495/.local/lib/python3.10/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/root495/.local/lib/python3.10/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/home/root495/.local/lib/python3.10/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global

{'medical_data': 'admission date:    2162-3-3    discharge date:    2162-3-25    date of birth:    2080-1-4    sex: m service: medicine al',
 'patient_notes': "17-year-old male, has come to the student health clinic complaining of heart pounding. mr. cleveland's mother has given ",
 'pmc_patients': 'this 60-year-old male was hospitalized due to moderate ards from covid-19 with symptoms of fever, dry cough, and dyspnea',
 'pubmed': ' biochemical studies on camomile components/iii. in vitro studies about the antipeptic activity of  -- -alpha-bisabolol '}

## Build Per-Document Inputs for Vector Storage

Instead of collapsing everything into a single mega-document, we keep each cleaned row as its own LangChain `Document`. This lets us track provenance per record and evaluate retrieval metrics such as precision@k / recall@k against specific document ids.


In [17]:
document_frames = []
for name, df in cleaned_dfs.items():
    df = df.copy()
    df['doc_id'] = df.apply(lambda row: f"{row['source']}::{int(row['source_row_id'])}", axis=1)
    cleaned_dfs[name] = df
    document_frames.append(df[['doc_id', 'source', 'source_row_id', 'clean_text']])

documents_df = pd.concat(document_frames, ignore_index=True)
print(f"Prepared {len(documents_df)} individual documents for downstream chunking")



Prepared 3744 individual documents for downstream chunking


,source,char_len
0,combined_corpus,12946090


In [ ]:
## Per-Document Chunking & Embeddings

Each cleaned row is wrapped as an individual LangChain `Document`, chunked with `RecursiveCharacterTextSplitter`, and embedded with `HuggingFaceEmbeddings` so retrieval metrics can be computed per `doc_id`.


SyntaxError: invalid character '’' (U+2019) (1300956446.py, line 3)

In [19]:

from langchain_core.documents import Document
import os

# Build LangChain Documents per cleaned record so we can evaluate retrieval per source row
documents = [
    Document(
        page_content=row['clean_text'],
        metadata={
            'source': row['source'],
            'source_row_id': int(row['source_row_id']),
            'doc_id': row['doc_id']
        }
    )
    for _, row in documents_df.iterrows()
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=100,
)

chunks = text_splitter.split_documents(documents)

# Track chunk indexes per originating document for downstream relevance metrics
chunk_records = []
doc_chunk_counts = defaultdict(int)
for chunk in chunks:
    doc_id = chunk.metadata['doc_id']
    chunk_index = doc_chunk_counts[doc_id]
    doc_chunk_counts[doc_id] += 1
    chunk.metadata['chunk_index'] = chunk_index

    chunk_records.append({
        'doc_id': doc_id,
        'source': chunk.metadata['source'],
        'source_row_id': chunk.metadata['source_row_id'],
        'chunk_index': chunk_index,
        'char_len': len(chunk.page_content)
    })

chunk_metadata_df = pd.DataFrame(chunk_records)

# Ensure the parent directory exists before saving
chunk_metadata_parent = os.path.dirname(str(CHUNK_METADATA_PATH))
if chunk_metadata_parent and not os.path.exists(chunk_metadata_parent):
    os.makedirs(chunk_metadata_parent, exist_ok=True)

chunk_metadata_df.to_parquet(CHUNK_METADATA_PATH, index=False)

print(f"Created {len(chunks)} chunks across {len(documents)} documents")
print(f"Chunk metadata saved to {CHUNK_METADATA_PATH}")



Created 6597 chunks across 3744 documents
Chunk metadata saved to /home/root495/Inexture/CDSS-RAG/notebooks/data/processed/chunk_metadata.parquet


In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
from langchain_community.vectorstores import Chroma

CHROMA_PERSIST_DIR.mkdir(parents=True, exist_ok=True)

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=str(CHROMA_PERSIST_DIR)
)

print(f"Chunks stored successfully in ChromaDB at {CHROMA_PERSIST_DIR}!")


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
from langchain_community.vectorstores import Chroma

# Load the existing ChromaDB vector store from the shared persistence directory
vectordb = Chroma(
    persist_directory=str(CHROMA_PERSIST_DIR),
    embedding_function=embedding_model
)


/tmp/ipykernel_40244/1948874181.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [6]:
query = "fever"  # your user's question

results = vectordb.similarity_search(query, k=5)

print(f"Retrieved {len(results)} chunks")


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Retrieved 5 chunks


In [7]:
for i, chunk in enumerate(results, 1):
    print(f"\n🟦 Chunk {i}")
    print(chunk)
    print("-" * 80)



🟦 Chunk 1
page_content='a 21-year-old male patient came to the hospital with complaints of fever, vomiting, pain in the abdomen, headache, and myalgia for three days. the patient was apparently alright three days ago when he started complaining of fever that was sudden in onset, intermittent, without diurnal variation, and not associated with chills or rigors. the fever was temporarily relived with antipyretics. he also had generalized weakness and joint pain associated with fever. nthe patient also complained of pain in the abdomen for three days. the pain was diffuse, present in the epigastrium and umbilical region, and aggravated after consumption of food. it was associated with nausea and vomiting. the patient had around four to five episodes of vomiting after food every day. vomitus contained food particles. there was no hematemesis. the patient was nonalcoholic. non examination, his pulse rate was 88 beats per minute, blood pressure was 110/80 mmhg in the right arm in the supine

In [ ]:
# Example: compute precision@k / recall@k against known document ids
# Assumptions:
# - `relevant_ids` contains the per-document `doc_id` values that should be surfaced
# - Retrieval results now expose `doc_id` + `chunk_index` metadata

def precision_recall_at_k(results, relevant_ids, k=5):
    if not results or not relevant_ids:
        return 0.0, 0.0

    retrieved_ids = [res.metadata.get("doc_id") for res in results[:k]]
    hits = [doc_id for doc_id in retrieved_ids if doc_id in relevant_ids and doc_id is not None]

    precision = len(hits) / k
    recall = len(hits) / len(relevant_ids)
    return precision, recall

relevant_ids = set([
    # e.g., "medical_data::42", "patient_notes::7"
])

k = 5
precision_at_k, recall_at_k = precision_recall_at_k(results, relevant_ids, k=k)

print(f"Precision@{k}: {precision_at_k:.3f}")
print(f"Recall@{k}: {recall_at_k:.3f}")


Top-1 id: None
Precision@1: 0.000
Recall@1: 0.000
